In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import warnings 
import warnings
warnings.filterwarnings('ignore')

In [2]:
ROOT_DIR_HIST = '/g/data/oi10/replicas/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/day/tas/gr/v20190614/'
hist = xr.open_dataset(ROOT_DIR_HIST + 'tas_day_IPSL-CM6A-LR_historical_r1i1p1f1_gr_18500101-20141231.nc'
                      , chunks = {'lat':100,'lon':100})

In [3]:
hist

<xarray.Dataset>
Dimensions:      (axis_nbounds: 2, lat: 143, lon: 144, time: 60265)
Coordinates:
  * lat          (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 87.46 88.73 90.0
  * lon          (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    height       float64 ...
  * time         (time) datetime64[ns] 1850-01-01T12:00:00 ... 2014-12-31T12:...
Dimensions without coordinates: axis_nbounds
Data variables:
    time_bounds  (time, axis_nbounds) datetime64[ns] dask.array<chunksize=(60265, 2), meta=np.ndarray>
    tas          (time, lat, lon) float32 dask.array<chunksize=(60265, 100, 100), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    creation_date:          2018-07-11T07:27:15Z
    description:            CMIP6 historical
    title:                  IPSL-CM6A-LR model output prepared for CMIP6 / CM...
    activity_id:            CMIP
    contact:                ipsl-cmip6@listes.ipsl.fr
    data_specs_version:     01.00.21
    dr2xml_version:         1.11
    experiment_id:          historical
    experiment:             all-forcing simulation of the recent past
    external_variables:     areacella
    forcing_index:          1
    frequency:              day
    grid:                   LMDZ grid
    grid_label:             gr
    nominal_resolution:     250 km
    initialization_index:   1
    institution_id:         IPSL
    institution:            Institut Pierre Simon Laplace, Paris 75252, France
    license:                CMIP6 model data produced by IPSL is licensed und...
    mip_era:                CMIP6
    parent_experiment_id:   piControl
    parent_mip_era:         CMIP6
    parent_activity_id:     CMIP
    parent_source_id:       IPSL-CM6A-LR
    parent_time_units:      days since 1850-01-01 00:00:00
    branch_method:          standard
    branch_time_in_parent:  21914.0
    branch_time_in_child:   0.0
    physics_index:          1
    product:                model-output
    realm:                  atmos
    source:                 IPSL-CM6A-LR (2017):  atmos: LMDZ (NPv6, N96; 144...
    source_id:              IPSL-CM6A-LR
    source_type:            AOGCM BGC
    sub_experiment_id:      none
    sub_experiment:         none
    table_id:               day
    variable_id:            tas
    EXPID:                  historical
    CMIP6_CV_version:       cv=6.2.3.5-2-g63b123e
    dr2xml_md5sum:          f1e40c1fc5d8281f865f72fbf4e38f9d
    model_version:          6.1.5
    parent_variant_label:   r1i1p1f1
    name:                   /ccc/work/cont003/gencmip6/p86caub/IGCM_OUT/IPSLC...
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.IPSL.IPSL-CM...
    variant_label:          r1i1p1f1
    realization_index:      1
    history:                Sat Dec  1 12:16:19 2018: ncatted -O -a realizati...
    NCO:                    "4.6.0"
    tracking_id:            hdl:21.14100/f03b9aba-9925-473a-b80a-522b166ed113

In [4]:
import os
ROOT_DIR = '/g/data/oi10/replicas/CMIP6/ScenarioMIP/IPSL/IPSL-CM6A-LR/'
ssp_scenarios = os.listdir(ROOT_DIR)
ssp_scenarios

['ssp460',
 'ssp585',
 'ssp126',
 'ssp119',
 'ssp534-over',
 'ssp245',
 'ssp434',
 'ssp370']

In [5]:
debug = 0

data_stor = {}
for scenario in ssp_scenarios:
    SUB_DIR = '/g/data/oi10/replicas/CMIP6/ScenarioMIP/IPSL/IPSL-CM6A-LR/{}/r1i1p1f1/Amon/tas/gr/'.\
    format(scenario)
    
    SCEN_DIR = SUB_DIR + os.listdir(SUB_DIR)[0]
    files = os.listdir(SCEN_DIR)
    
    # Printing out the full list of what is occuring
    if debug:
        print(SCEN_DIR)
        print(files, sep = '\n')
        print(len(files))
        print('------')
    
    # Certain scerious contain either one data set (run to 2100) or two data sets (runs to 2300).
    # If there is only data set we can open this plainly.
    if len(files) > 1:
        df  = xr.open_mfdataset(SCEN_DIR + '/*.nc', use_cftime = True, 
                                chunks = {'lat':100,'lon':100})
    # If there are two, we can employ a mutilfile merge to get them into the one dataset.
    else:
        df = xr.open_dataset(SCEN_DIR + '/{}'.format(files[0]), use_cftime = True, 
                             chunks = {'lat':100,'lon':100})
    
    print('{} has been loaded ({} files)'.format(scenario, len(files)))
    
    data_stor[scenario] = df

ssp460 has been loaded (1 files)
ssp585 has been loaded (2 files)
ssp126 has been loaded (2 files)
ssp119 has been loaded (1 files)
ssp534-over has been loaded (2 files)
ssp245 has been loaded (1 files)
ssp434 has been loaded (1 files)
ssp370 has been loaded (1 files)


In [6]:
IPSL = xr.concat(data_stor.values(), pd.Index( data_stor.keys(), name = 'scenario'))\
            .chunk({'lat':100,'lon':100})
IPSL

<xarray.Dataset>
Dimensions:      (axis_nbounds: 2, lat: 143, lon: 144, scenario: 8, time: 3432)
Coordinates:
  * time         (time) object 2015-01-16 12:00:00 ... 2300-12-16 12:00:00
  * lat          (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 87.46 88.73 90.0
  * lon          (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    height       float64 2.0
  * scenario     (scenario) object 'ssp460' 'ssp585' ... 'ssp434' 'ssp370'
Dimensions without coordinates: axis_nbounds
Data variables:
    time_bounds  (scenario, time, axis_nbounds) object dask.array<chunksize=(1, 1032, 2), meta=np.ndarray>
    tas          (scenario, time, lat, lon) float32 dask.array<chunksize=(1, 1032, 100, 100), meta=np.ndarray>
Attributes:
    name:                   /ccc/work/cont003/gencmip6/lurtont/IGCM_OUT/IPSLC...
    Conventions:            CF-1.7 CMIP-6.2
    creation_date:          2019-04-10T06:43:43Z
    tracking_id:            hdl:21.14100/3b7212e9-b7cf-4170-9c1f-932c27c60b60
    description:            Future scenario with medium radiative forcing by ...
    title:                  IPSL-CM6A-LR model output prepared for CMIP6 / Sc...
    activity_id:            ScenarioMIP
    contact:                ipsl-cmip6@listes.ipsl.fr
    data_specs_version:     01.00.28
    dr2xml_version:         1.16
    experiment_id:          ssp460
    experiment:             update of RCP6.0 based on SSP4
    external_variables:     areacella
    forcing_index:          1
    frequency:              mon
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.IPSL.IPSL-CM...
    grid:                   LMDZ grid
    grid_label:             gr
    nominal_resolution:     250 km
    history:                none
    initialization_index:   1
    institution_id:         IPSL
    institution:            Institut Pierre Simon Laplace, Paris 75252, France
    license:                CMIP6 model data produced by IPSL is licensed und...
    mip_era:                CMIP6
    parent_mip_era:         CMIP6
    parent_source_id:       IPSL-CM6A-LR
    parent_time_units:      days since 1850-01-01 00:00:00
    parent_variant_label:   r1i1p1f1
    branch_method:          standard
    branch_time_in_parent:  60265.0
    branch_time_in_child:   0.0
    physics_index:          1
    product:                model-output
    realization_index:      1
    realm:                  atmos
    source:                 IPSL-CM6A-LR (2017):  atmos: LMDZ (NPv6, N96; 144...
    source_id:              IPSL-CM6A-LR
    source_type:            AOGCM BGC
    sub_experiment_id:      none
    sub_experiment:         none
    table_id:               Amon
    variable_id:            tas
    variant_info:           Each member starts from the corresponding member ...
    variant_label:          r1i1p1f1
    EXPID:                  ssp460
    CMIP6_CV_version:       cv=6.2.15.1
    dr2xml_md5sum:          b6f602401512e82e2d7cadc2c6f36c2a
    model_version:          6.1.9
    parent_experiment_id:   historical
    parent_activity_id:     CMIP

In [14]:
class Complex:
    
    def __init__(self, r, i):
        self.r = r
        self.i = i

In [18]:
x = Complex(3,1)

In [20]:
x.r

3

In [19]:
x.i

1

In [26]:
class Dog:
    
    def __init__(self, name):
        self.name = name
        self.tricks = []
    
    def add_trick(self, trick):
        self.tricks.append(trick)
                

In [27]:
d = Dog('Fido')

In [28]:
d.name

'Fido'

In [33]:
d.add_trick('alex')

In [44]:
class Reverse:
    def __init__(self, data):
        self.data = data
        self.index = len(data)
        
    def __iter__(self):
        return self
    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]

In [45]:
rev = Reverse('Alex')

In [46]:
for char in rev:
    print(char)

x
e
l
A


In [47]:
rev.data

'Alex'

In [48]:
rev.index

0

In [59]:
class Polynomial(object):
    explanation = 'polynomlai class'
    
    def explain(self):
        print(self.explanation)

In [60]:
p = Polynomial()

In [61]:
p.explanation

'polynomlai class'

In [62]:
p.explain()

polynomlai class


In [63]:
p.explenation = 'alex'

In [64]:
p.explain()

polynomlai class


In [65]:
class Polynomial(object):
    explanation = 'poly object'
    
    def explain_to(self, caller):
        print('Hell {} {}'.format(self.explanation, caller))

In [67]:
r = Polynomial()

In [68]:
r.explanation

'poly object'

In [69]:
r.explain_to('Alex')

Hell poly object Alex


In [80]:
class Polynomial(object):
    explanation = 'poly obj'
    
    def __init__(self, roots, leading_term):
        self.roots = roots
        self.leading_term = leading_term
        self.order = len(roots)
        
    def explain_to(self, caller):
        print('Hello {} {}'.format(caller, self.explanation))
        print('My Roots are {}'.format(self.roots))
    

In [82]:
q = Polynomial((1,1,1,1,1,1,0,-2), -1)
q.explain_to("Bob")

Hello Bob poly obj
My Roots are (1, 1, 1, 1, 1, 1, 0, -2)


In [84]:
q.order

8

In [86]:
class XarrayClass(object):
    def subtract_data(self, data, dim, value):
        data[dim] -= value
    

In [89]:
r = XarrayClass()

In [90]:
hist

<xarray.Dataset>
Dimensions:      (axis_nbounds: 2, lat: 143, lon: 144, time: 60265)
Coordinates:
  * lat          (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 87.46 88.73 90.0
  * lon          (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    height       float64 ...
  * time         (time) datetime64[ns] 1850-01-01T12:00:00 ... 2014-12-31T12:...
Dimensions without coordinates: axis_nbounds
Data variables:
    time_bounds  (time, axis_nbounds) datetime64[ns] dask.array<chunksize=(60265, 2), meta=np.ndarray>
    tas          (time, lat, lon) float32 dask.array<chunksize=(60265, 100, 100), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    creation_date:          2018-07-11T07:27:15Z
    description:            CMIP6 historical
    title:                  IPSL-CM6A-LR model output prepared for CMIP6 / CM...
    activity_id:            CMIP
    contact:                ipsl-cmip6@listes.ipsl.fr
    data_specs_version:     01.00.21
    dr2xml_version:         1.11
    experiment_id:          historical
    experiment:             all-forcing simulation of the recent past
    external_variables:     areacella
    forcing_index:          1
    frequency:              day
    grid:                   LMDZ grid
    grid_label:             gr
    nominal_resolution:     250 km
    initialization_index:   1
    institution_id:         IPSL
    institution:            Institut Pierre Simon Laplace, Paris 75252, France
    license:                CMIP6 model data produced by IPSL is licensed und...
    mip_era:                CMIP6
    parent_experiment_id:   piControl
    parent_mip_era:         CMIP6
    parent_activity_id:     CMIP
    parent_source_id:       IPSL-CM6A-LR
    parent_time_units:      days since 1850-01-01 00:00:00
    branch_method:          standard
    branch_time_in_parent:  21914.0
    branch_time_in_child:   0.0
    physics_index:          1
    product:                model-output
    realm:                  atmos
    source:                 IPSL-CM6A-LR (2017):  atmos: LMDZ (NPv6, N96; 144...
    source_id:              IPSL-CM6A-LR
    source_type:            AOGCM BGC
    sub_experiment_id:      none
    sub_experiment:         none
    table_id:               day
    variable_id:            tas
    EXPID:                  historical
    CMIP6_CV_version:       cv=6.2.3.5-2-g63b123e
    dr2xml_md5sum:          f1e40c1fc5d8281f865f72fbf4e38f9d
    model_version:          6.1.5
    parent_variant_label:   r1i1p1f1
    name:                   /ccc/work/cont003/gencmip6/p86caub/IGCM_OUT/IPSLC...
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.IPSL.IPSL-CM...
    variant_label:          r1i1p1f1
    realization_index:      1
    history:                Sat Dec  1 12:16:19 2018: ncatted -O -a realizati...
    NCO:                    "4.6.0"
    tracking_id:            hdl:21.14100/f03b9aba-9925-473a-b80a-522b166ed113

In [93]:
test = r.subtract_data(hist, 'tas', 3)

In [94]:
test

r.subtract_data(hist, 'tas

# Mapblocks

This is from scott's video on analysing big data sets with xarray.

https://www.youtube.com/watch?v=WIPtMpD85tI

In [107]:
def dask_percentile(array, axis, q):
    array = array.rechunk({axis:-1})
    
    return array.map_blocks(
        np.percentile,
        axis = axis,
        q = q,
        dtype = array.dtype,
        drop_axis = axis
    )

In [108]:
hist.reduce(dask_percentile, dim = 'time', q = q)

<xarray.Dataset>
Dimensions:      (axis_nbounds: 2, lat: 143, lon: 144)
Coordinates:
  * lat          (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 87.46 88.73 90.0
  * lon          (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    height       float64 ...
Dimensions without coordinates: axis_nbounds
Data variables:
    time_bounds  (axis_nbounds) datetime64[ns] dask.array<chunksize=(2,), meta=np.ndarray>
    tas          (lat, lon) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>

In [116]:
import bottleneck
import xarray.ufuncs as xu

In [111]:
def temp_str(array):

    if temp < 3:
        return 'cold'
    return 'hot'

In [113]:
hist.tas.apply_ufuncs(temp_str)

AttributeError: 'DataArray' object has no attribute 'apply_ufuncs'

In [114]:
xr.appy_ufunc

AttributeError: module 'xarray' has no attribute 'appy_ufunc'

In [117]:
def covariance_gufunc(x, y):
    return (
        (x - x.mean(axis=-1, keepdims=True)) * (y - y.mean(axis=-1, keepdims=True))
    ).mean(axis=-1)


def pearson_correlation_gufunc(x, y):
    return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


def spearman_correlation_gufunc(x, y):
    x_ranks = bottleneck.rankdata(x, axis=-1)
    y_ranks = bottleneck.rankdata(y, axis=-1)
    return pearson_correlation_gufunc(x_ranks, y_ranks)


def spearman_correlation(x, y, dim):
    return xr.apply_ufunc(
        spearman_correlation_gufunc,
        x,
        y,
        input_core_dims=[[dim], [dim]],
        dask="parallelized",
        output_dtypes=[float],
    )

In [121]:
bottleneck.rankdata([1,2,3,4,5,5,34,453,234,23,4,41,4,4])

array([ 1. ,  2. ,  3. ,  5.5,  8.5,  8.5, 11. , 14. , 13. , 10. ,  5.5,
       12. ,  5.5,  5.5])

In [118]:
spearman_correlation(hist.tas, hist.tas, dim = 'time')

<xarray.DataArray 'tas' (lat: 143, lon: 144)>
dask.array<transpose, shape=(143, 144), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 86.2 87.46 88.73 90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    height   float64 2.0

In [122]:
def func(da):
    print(da.sizes)
    return da.time

In [124]:
mapped = xr.map_blocks(func, hist.tas)

Frozen({'time': 0, 'lat': 0, 'lon': 0})


In [125]:
mapped

<xarray.DataArray 'time' (time: 60265)>
dask.array<func-27e354baae245d727395d3b2a060c875-<this, shape=(60265,), dtype=datetime64[ns], chunksize=(60265,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1850-01-01T12:00:00 ... 2014-12-31T12:00:00
    height   float64 dask.array<chunksize=(), meta=np.ndarray>
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    time_origin:    1850-01-01 00:00:00
    bounds:         time_bounds

Following examples are from:
http://xarray.pydata.org/en/stable/generated/xarray.map_blocks.html

In [127]:
def caculate_anomaly(da, groupby_type = 'time.month'):
    gb = da.groubpy(groupby_type)
    clim = gb.mean(dim = 'time')
    return gb - clim

In [129]:
time = xr.cftime_range("1990-01", "1992-01", freq="M")
month = xr.DataArray(time.month, coords={"time": time}, dims=["time"])
np.random.seed(123)

array = xr.DataArray(
            np.random.rand(len(time)),
            dims=["time"],
            coords={"time": time, "month": month},
        ).chunk()
array

<xarray.DataArray (time: 24)>
dask.array<xarray-<this-array>, shape=(24,), dtype=float64, chunksize=(24,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1990-01-31 00:00:00 ... 1991-12-31 00:00:00
    month    (time) int64 dask.array<chunksize=(24,), meta=np.ndarray>

In [131]:
array.map_blocks(caculate_anomaly, template = array)

<xarray.DataArray (time: 24)>
dask.array<caculate_anomaly-8edfea6b270ad469cd49c2664f502e74-<this, shape=(24,), dtype=float64, chunksize=(24,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1990-01-31 00:00:00 ... 1991-12-31 00:00:00
    month    (time) int64 dask.array<chunksize=(24,), meta=np.ndarray>

In [135]:
def temp_str(temp):

    if temp < 3:
        return 'cold'
    return 'hot'

In [136]:
array.map_blocks(temp_str, template = array).compute()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

http://xarray.pydata.org/en/stable/examples/apply_ufunc_vectorize_1d.html